<a href="https://colab.research.google.com/github/wakamatsuikuma/MEMO_causal_analysis_by_python/blob/main/7_2_%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E3%81%AE%E5%BD%93%E3%81%A6%E3%81%AF%E3%81%BE%E3%82%8A%E3%81%AE%E8%89%AF%E3%81%95%E3%82%92%E6%B8%AC%E3%82%8B%E6%96%B9%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7.2 BICの計算  
データに対して、ネットワークのモデルを与え(=条件付き確率表を推定)、BICを計算する。与えるモデルのBICを比較することで、どのモデルが当てはまりが良いかを求められる。(たぶん因果の向きを推定できるってことのように思う。)  
  
1. モデルを与える(因果の有無、因果の方向性だけ示す)
2. 条件付き確率表が推定できる
3. BICを計算できる=どのモデルが妥当か推定できる=因果の向きがわかる(?) 


コーディングのイメージ
1. BayesianModel()で因果モデルを与える（変数(カラム)を指定。）  
2. BicScore(df)でデータセット指定  
3. BIcScore(df).score(BayesianModel())でスコア計算(データセットと因果モデルが与えられて、CPTと情報量基準スコアが得られる)  

（事前分布：経験則なりなんなりで事前に設定、尤度関数=条件付き確率：取得データや与えるモデルで決まる）

# プログラム実行前の設定など

In [ ]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

In [ ]:
# 使用するパッケージ（ライブラリと関数）を定義
from numpy.random import *
import pandas as pd

# データの生成

In [ ]:
# データ数
num_data = 100

# x1：0か1の値をnum_data個生成、0の確率は0.6、1の確率は0.4
x1 = np.random.choice([0, 1], num_data, p=[0.6, 0.4])

# x2：0か1の値をnum_data個生成、0の確率は0.4、1の確率は0.6
x2 = np.random.choice([0, 1], num_data, p=[0.4, 0.6])

# 2変数で表にする
df = pd.DataFrame({'x1': x1,
                   'x2': x2,
                   })

df.head()  # 先頭を表示


,x1,x2
0,1,0
1,0,1
2,1,1
3,0,0
4,1,0


In [ ]:
# 変数x3：0か1の値をnum_data個生成する
# (x1,x2)= (0,0)のとき、0の確率は0.2
# (x1,x2)= (1,0)のとき、0の確率は0.3
# (x1,x2)= (0,1)のとき、0の確率は0.4
# (x1,x2)= (1,1)のとき、0の確率は0.1

x3 = []
for i in range(num_data):
  if x1[i] == 0 and x2[i] == 0:
    x3_value = np.random.choice([0, 1], 1, p=[0.2, 0.8])
    x3.append(x3_value[0])  # x3はリストになっているので、0番目の要素を取り出して追加
  elif x1[i] == 0 and x2[i] == 1:
    x3_value = np.random.choice([0, 1], 1, p=[0.3, 0.7])
    x3.append(x3_value[0])
  elif x1[i] == 1 and x2[i] == 0:
    x3_value = np.random.choice([0, 1], 1, p=[0.4, 0.6])
    x3.append(x3_value[0])
  elif x1[i] == 1 and x2[i] == 1:
    x3_value = np.random.choice([0, 1], 1, p=[0.1, 0.9])
    x3.append(x3_value[0])

df["x3"] = x3

df  # 表示

,x1,x2,x3
0,1,0,0
1,0,1,1
2,1,1,1
3,0,0,1
4,1,0,0
...,...,...,...
95,0,1,0
96,0,0,1
97,1,0,0
98,0,0,0


- num_data=10では、x1はs出現頻度が逆になるくらいには微妙なデータになっている

# pgmpy(Python library for Probabilistic Graphical Models)によるBICの計算

In [ ]:
!pip install pgmpy==0.1.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 正解のDAGを与える
from pgmpy.models import BayesianModel
model = BayesianModel([('x1', 'x3'), ('x2', 'x3')]) # x1-> x3 <- x2

In [ ]:
# 各データパターンの個数を表示する
from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, df)

display(pe.state_counts('x1'))

print("\n")
display(pe.state_counts('x2'))

print("\n")
display(pe.state_counts('x3'))

,x1
0,59
1,41


,x2
0,37
1,63


x1     0          1      
x2     0     1    0     1
x3                       
0    5.0  14.0  5.0   0.0
1   22.0  18.0  5.0  31.0

In [ ]:
# CPTを推定して出力する
## ここではデータが多項分布に従い、データの生成確率はすべてハイパーパラメータ0のディリクレ分布を仮定する
from pgmpy.estimators import BayesianEstimator

estimator = BayesianEstimator(model, df)

cpd_x1 = estimator.estimate_cpd('x1', prior_type='dirichlet', 
                                pseudo_counts=[[0], [0]])
cpd_x2 = estimator.estimate_cpd('x2', prior_type='dirichlet', 
                                pseudo_counts=[[0], [0]])
cpd_x3 = estimator.estimate_cpd('x3', prior_type='dirichlet', 
                                pseudo_counts=[[0, 0, 0, 0], [0, 0, 0, 0]])

print(cpd_x1)
print(cpd_x2)
print(cpd_x3)

+-------+------+
| x1(0) | 0.59 |
+-------+------+
| x1(1) | 0.41 |
+-------+------+
+-------+------+
| x2(0) | 0.37 |
+-------+------+
| x2(1) | 0.63 |
+-------+------+
+-------+---------------------+--------+-------+-------+
| x1    | x1(0)               | x1(0)  | x1(1) | x1(1) |
+-------+---------------------+--------+-------+-------+
| x2    | x2(0)               | x2(1)  | x2(0) | x2(1) |
+-------+---------------------+--------+-------+-------+
| x3(0) | 0.18518518518518517 | 0.4375 | 0.5   | 0.0   |
+-------+---------------------+--------+-------+-------+
| x3(1) | 0.8148148148148148  | 0.5625 | 0.5   | 1.0   |
+-------+---------------------+--------+-------+-------+


In [ ]:
# BICを求める
# BayesianModel()で因果モデルを与える（変数(カラム)を指定。）
# BicScore(df)でデータセット指定
# BIcScore(df).score(BayesianModel())でスコア計算(データセットと因果モデルが与えられて、CPTと情報量基準スコアが得られる)

from pgmpy.estimators import BicScore
bic = BicScore(df)
print(bic.score(model))

-189.19593158427529


#  異なるDAGとの比較

In [ ]:
from pgmpy.models import BayesianModel

model2 = BayesianModel([('x2', 'x1'), ('x2', 'x3')]) # x1 <- x2 -> x3
bic2 = BicScore(df)
print(bic2.score(model2))

-197.62135251950247


- num_data = 10  
model:-21.65605747450808  
model2:-21.425819218840655  
  
- num_data = 100  
model:-189.19593158427529  
model2:-197.62135251950247  
  
データ数が増えれば、正しいDAGが当てはまりが良くなった